In [ ]:
# Verify which python environment we are running the notebook on
# !pyenv version
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install nltk
#%pip install --upgrade pip


In [ ]:
# Import packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json


In [ ]:
# Retrieve the data from csv unzipped files into two dataframes
#wine1_df=pd.read_csv("~/code/sabrinaauger/wino/raw_data/winemag-data-130k-v2.csv")
wine1_df=pd.read_csv("~/code/sabrinaauger/wino/data/raw_data/winemag-data-130k-v2.csv")
#wine2_df=pd.read_csv("~/code/sabrinaauger/wino/raw_data/winemag-data_first150k.csv")
wine2_df=pd.read_csv("~/code/sabrinaauger/wino/data/raw_data/winemag-data_first150k.csv")


In [ ]:
wine1_df.head()


In [ ]:
# Importing
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

wine1_df['description_tokenized'] = wine1_df['description'].apply(word_tokenize)
wine1_df.head()


In [ ]:
# Word2Vec from https://code.google.com/archive/p/word2vec/
model = Word2Vec(sentences=wine1_df['description_tokenized'], vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
#
def get_sentence_vector(sentence, model):
    words_in_vocab = [word for word in sentence if word in model.wv.key_to_index]

    # If no words are in the vocabulary, then it becomes 0
    if not words_in_vocab:
        return [0] * model.vector_size

    # Calculate the mean vector for the whole sentence
    vector = sum(model.wv[word] for word in words_in_vocab) / len(words_in_vocab)
    return vector

wine1_df['description_vectorized'] = wine1_df['description_tokenized'].apply(lambda x: get_sentence_vector(x, model))


wine1_df.head()


In [ ]:
wine1_df.head(2)


In [ ]:
# Sorting which feature is what
y = ['designation']
categorical_features_list = ['country', 'province', 'region_1', 'region_2', 'variety', 'winery']
text_features_list = ['taster_name', 'taster_twitter_handle', 'title']
numerical_features_list = ['points', 'price']
# country	description	designation	points	price	province	region_1	region_2	taster_name	taster_twitter_handle	title	variety	winery


In [ ]:

# Vectorizing numeric features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(wine1_df[numerical_features_list])
wine1_df_scaled = pd.concat([wine1_df, pd.DataFrame(numeric_scaled, columns=['points_scaled', 'price_scaled'])], axis=1)
#wine1_df_scaled.head()


In [ ]:
# Vectorizing categorical features
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

category_encoded = encoder.fit_transform(wine1_df[categorical_features_list]).toarray()
wine1_df_scaled = pd.concat([wine1_df, pd.DataFrame(category_encoded, columns=encoder.get_feature_names_out(categorical_features_list))], axis=1)
wine1_df_scaled.head(2)
